In [1]:
from tqdm import tqdm
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import AutoTokenizer, AutoModelWithLMHead

In [9]:
import torch
from sklearn import preprocessing
import itertools

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
from transformers import AutoConfig
config = AutoConfig.from_pretrained("bert-base-uncased")
# not sure this works
config.architectures[0]

'BertForMaskedLM'

In [14]:
data= load_dataset("poem_sentiment","plain_text", split= "test", streaming= True)

Using custom data configuration plain_text


In [22]:
["oscar", "unshuffled_deduplicated_en", "train", True,5],
        ["imdb","plain_text", "test", False ,5],
        ["glue","ax", "test", True , 5],
        ["poem_sentiment","plain_text", "test", True ,5],
        
        
        
        "bert-base-uncased"

['verse_text']

In [42]:
mods= ["t5-small"]

datas= [
        
        ["c4", "en", "train", True, 5]
        
       ] 

In [ ]:
perplex_model_data(m_name ="bert-base-uncased", d_name ="oscar", d_option="unshuffled_deduplicated_en", d_split = "train", d_streaming = True, d_size=5 )   

In [47]:
def perplex_model_data(**kwargs):
    modlist=[]
    maskedLM= ["bert-base-uncased"]
    maskedHead= ["t5-small"]
    tok = AutoTokenizer.from_pretrained(kwargs['m_name'])
    if kwargs['m_name'] in modlist: 
        pass
    elif kwargs['m_name'] in maskedHead:
        model = AutoModelWithLMHead.from_pretrained(kwargs['m_name'])
    else: 
        model = AutoModelForMaskedLM.from_pretrained(kwargs['m_name'])
        print(str(kwargs['m_name']) + " model loaded!")
        modlist.append(kwargs['m_name'])
    data= load_dataset(kwargs['d_name'], kwargs['d_option'], split= kwargs['d_split'], streaming = kwargs['d_streaming'])
    print(str(kwargs['d_name']) + " dataset loaded!")
    if kwargs['d_streaming'] == True: 
        data_head = data.take(kwargs['d_size'])
        try: 
            text= [l['text'] for l in list(data_head)]
        except:
            feature = [d for d in data.features if 'text' in d]
            feature=feature[0]
            text= [l[feature] for l in list(data_head)]
        
    else:
        feature = next(iter(data.features))
        text= data[feature][:kwargs['d_size']]
    
    encodings = tok('\n\n'.join(text), return_tensors='pt')
    try:
        max_length = model.config.max_position_embeddings
    except AttributeError as error:
        max_length = model.config.n_positions
    except:
        max_length=512
        
    #From https://huggingface.co/transformers/perplexity.html
    stride = 512
    lls = []
    for i in range(0, encodings.input_ids.size(1), stride):
        begin_loc = max(i + stride - max_length, 0)
        end_loc = min(i + stride, encodings.input_ids.size(1))
        trg_len = end_loc - i    # may be different from stride on last loop
        input_ids = encodings.input_ids[:,begin_loc:end_loc].to(device)
        target_ids = input_ids.clone()
        target_ids[:,:-trg_len] = -100

        with torch.no_grad():
            outputs = model(input_ids, labels=target_ids)
            log_likelihood = outputs[0] * trg_len

        lls.append(log_likelihood)

    ppl = torch.exp(torch.stack(lls).sum() / end_loc)
    print("The perplexity of the " + str(kwargs['m_name']) + " model with the " + str(kwargs['d_name']) + " dataset is " + str(ppl.item()))

In [43]:
def get_combo_pp(models, datasets):
    combos= list(itertools.product(models, datasets))
    print("There are " + str(len(list(combos))) + " combinations of models and datasets.")
    for m, d in combos:
        print('Analyzing the ' + m + ' model and the ' + d[0] + ' dataset.')
        perplex_model_data(m_name= m, d_name= d[0], d_option= d[1], d_split= d[2], d_streaming= d[3], d_size= d[4])

In [ ]:
get_combo_pp(mods,datas)

There are 1 combinations of models and datasets.
Analyzing the t5-small model and the c4 dataset.


In [ ]:
from datasets import load_dataset
base_url = "https://storage.googleapis.com/huggingface-nlp/cache/datasets/wikipedia/20200501.en/1.0.0/"
data_files = {"train": base_url + "wikipedia-train.parquet"}
wiki = load_dataset("parquet", data_files=data_files, split="train", streaming=True)
print(next(iter(wiki)))
# {'title': 'Yangliuqing', 'text': 'Yangliuqing () is a market town in Xiqing District...'}